In [1]:
import numpy as np
import pymc3 as pm
import theano
import theano.tensor as tt

In [2]:
with pm.Model() as model:
    x = pm.Normal('x', mu=0, sd=3)

    a = theano.shared(np.array([1.,2.,3.,4.,5.]))
    b = a + x
    c = a[(b > 5)]
    s = tt.sum(c)

    sigma = pm.HalfCauchy('sigma', 2.5)
    y_obs = pm.Normal("s", mu=s, sd=sigma, observed=15)

#     trace = pm.sample(500)

In [3]:
with model:
    step = pm.HamiltonianMC()

In [4]:
step.integrator._logp_dlogp_func.set_extra_values({})

In [5]:
model.test_point

{'x': array(0.), 'sigma_log__': array(0.91629073)}

In [6]:
step.integrator._logp_dlogp_func.dict_to_array(model.test_point)

array([0.91629073, 0.        ])

In [7]:
step.integrator._logp_dlogp_func(np.array([0.91629073, 0.]))

(array(-22.99751006), array([35.00000014,  0.        ]))

In [8]:
step.integrator._logp_dlogp_func(np.array([0.91629073, 4.]))

(array(-5.96639889), array([-0.84      , -0.44444444]))

In [9]:
step.integrator._logp_dlogp_func(np.array([0.91629073, -1.]))

(array(-23.05306562), array([35.00000014,  0.11111111]))

In [11]:
import tensorflow as tf
import tensorflow_probability as tfp

In [12]:
a = tf.constant([1.,2.,3.,4.,5.])
x = tf.constant([3.])
# x = tf.get_variable("x1", initializer=tf.constant([3.]))
b = a + x
c = tf.boolean_mask(a, b>5)
s = tf.reduce_sum(c)
ga = tf.gradients(s, [a])
gx = tf.gradients(s, [x])

with tf.Session() as sess:
    print(sess.run(s))
    print(sess.run(ga))
    print(sess.run(gx)) # This fail, same for a variable x.

12.0
[array([0., 0., 1., 1., 1.], dtype=float32)]


TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [8]:
tf.reset_default_graph()
a = tf.constant([1., 2., 3., 4., 5.])
x = tf.get_variable("x", initializer=tf.constant(3.))
sigma = tf.get_variable("sigma", initializer=tf.constant(0.1))

b = a + x
c = tf.boolean_mask(a, b > 5)
s = tf.reduce_sum(c)

loglike = tfd.Normal(loc=s, scale=sigma).log_prob(15.) + tfd.HalfCauchy(
    loc=0., scale=2.5).log_prob(sigma) + tfd.Normal(loc=0., scale=3.).log_prob(x)

ga = tf.gradients(loglike, [a])
gx = tf.gradients(loglike, [x])

with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(s))
    print(sess.run(ga))
    print(sess.run(gx))

12.0
[array([  0.,   0., 300., 300., 300.], dtype=float32)]
[-0.33333337]


In [10]:
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(gx, feed_dict={x: 1.}))
    print(sess.run(gx, feed_dict={x: 2.}))

[-0.11111112]
[-0.22222224]


In [4]:
loglike2 = tfd.Normal(loc=s, scale=sigma).log_prob(15.)
ga = tf.gradients(loglike2, [a])
prior_x = tfd.Normal(loc=0., scale=3.).log_prob(x)
gx_p = tf.gradients(prior_x, [x])
gx = tf.gradients(loglike2, [x])

with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(s))
    print(sess.run(ga))
    print(sess.run(gx_p))
    print(sess.run(gx))

12.0
[array([  0.,   0., 300., 300., 300.], dtype=float32)]
[-0.33333337]


TypeError: Fetch argument None has invalid type <class 'NoneType'>